In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ishworsubedii/nepali-speech-to-text-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.01G/1.01G [00:09<00:00, 119MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ishworsubedii/nepali-speech-to-text-dataset/versions/1


In [ ]:
!mv /root/.cache/kagglehub/datasets/ishworsubedii/nepali-speech-to-text-dataset/versions/1 /content/drive/MyDrive

In [ ]:
!wget https://huggingface.co/datasets/spktsagar/openslr-nepali-asr-cleaned/resolve/main/openslr-nepali-asr-cleaned.py

Could not load dataset directly: Dataset scripts are no longer supported, but found openslr-nepali-asr-cleaned.py
Attempting to load from local path if available...


In [ ]:
from datasets import load_dataset, concatenate_datasets

nepali = load_dataset("amitpant7/nepali-speech-to-text", split="train")



Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="multilingual", task="transcribe")

def prepare(example):
    audio = example["audio"]
    example["input_features"] = processor(audio["array"], sampling_rate=16000).input_features[0]
    example["labels"] = processor.tokenizer(example["transcription"]).input_ids # Corrected from example["sentence"]
    return example

dataset = nepali.map(prepare, remove_columns=['audio', 'transcription'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [2]:
!apt-get update && apt-get install -y ffmpeg  # Fix for TorchCodec/FFmpeg issues
!pip install -q datasets==2.21.0 accelerate peft bitsandbytes jiwer evaluate librosa soundfile transformers

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,856 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy In

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load the dataset (it has multiple 'train.*' splits)
nepali = load_dataset("amitpant7/nepali-speech-to-text")

# Concatenate all train splits (ignore any test if present)
train_splits = [split for split in nepali if 'train' in split]
nepali_train = concatenate_datasets([nepali[split] for split in train_splits])

print(f"Loaded {len(nepali_train)} training samples.")
print(nepali_train)
print(nepali_train[0])  # Preview one example

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Loaded 2650 training samples.
Dataset({
    features: ['audio', 'transcription'],
    num_rows: 2650
})
{'audio': {'path': 'train_audio_0.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        2.60547258e-05,  8.18362423e-06, -4.72624306e-05]), 'sampling_rate': 32000}, 'transcription': 'म पनि जान्छु है त अहिले लाई ।'}


In [ ]:
from transformers import WhisperProcessor
import torch

# Load processor with Nepali language
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    task="transcribe"
)

def prepare(example):
    audio = example["audio"]
    # Resample audio to 16000 Hz
    audio = audio.copy_resample(16000)
    # Use actual sampling rate for resampling; return PyTorch tensors
    example["input_features"] = processor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt"
    ).input_features[0]

    # Encode transcription to labels
    example["labels"] = processor.tokenizer(example["transcription"]).input_ids
    return example

# Map safely: single process, non-batched to avoid OOM
full = full.map(
    prepare,
    remove_columns=nepali_train.column_names,  # Clean up after processing
    num_proc=1,
    batched=False
)

# Optional: Add a simple validation split (10% of train)
nepali_train = nepali_train.train_test_split(test_size=0.1, shuffle=True)
print("Processed dataset splits:", nepali_train)

# Save to disk if needed (for resuming)
nepali_train.save_to_disk("/content/drive/MyDrive/nepali_whisper_processed")

In [ ]:
!pip install --upgrade transformers


In [ ]:
from datasets import load_from_disk
import torch
from transformers import WhisperProcessor
from peft import LoraConfig, get_peft_model, TaskType

# Load saved dataset (includes train/test splits)
nepali_train = load_from_disk("/content/drive/MyDrive/nepali_whisper_processed")
print("Loaded dataset:", nepali_train)
print(f"Train samples: {len(nepali_train['train'])}, Eval samples: {len(nepali_train['test'])}")

# Reload processor (for collator and metrics)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language="ne",
    task="transcribe"
)

# Your compute_metrics function (from previous cell)
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Loaded dataset: DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2385
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 265
    })
})
Train samples: 2385, Eval samples: 265


In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# -----------------------------
# CORRECTED Data collator for Whisper
# -----------------------------
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # input_features already processed, just stack them
        input_features = [torch.tensor(f["input_features"]) for f in features]

        # Pad input features to same length
        input_features = torch.nn.utils.rnn.pad_sequence(
            input_features, batch_first=True, padding_value=0.0
        )

        # Handle labels
        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch = {
            "input_features": input_features,
            "labels": labels,
        }
        return batch

# -----------------------------
# Training parameters
# -----------------------------
model_id = "small"
task = "transcribe"
language = "ne"

# LoRA parameters
r = 16
lora_alpha = 32
lora_dropout = 0.1

# Training parameters
learning_rate = 1e-4
train_batch_size = 8  # Reduced to avoid memory issues
gradient_accumulation_steps = 2
fp16 = True
warmup_steps = 100
max_steps = 1000
save_steps = 200
logging_steps = 50

experiment_name = f"whisper-{model_id}-lora-nepali"

# -----------------------------
# Load processor and model
# -----------------------------
processor = WhisperProcessor.from_pretrained(
    f"openai/whisper-{model_id}",
    language=language,
    task=task
)

# Load model
model = WhisperForConditionalGeneration.from_pretrained(
    f"openai/whisper-{model_id}",
    device_map="auto"
)

# Configure model
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False
model.gradient_checkpointing_enable()

# -----------------------------
# Apply LoRA
# -----------------------------
peft_config = LoraConfig(
    r=r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=lora_dropout,
    bias="none",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# -----------------------------
# Data collator
# -----------------------------
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# -----------------------------
# Training arguments
# -----------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir=f"/content/drive/MyDrive/{experiment_name}",
    per_device_train_batch_size=train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    max_steps=max_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    eval_steps=200,
    save_total_limit=3,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=["tensorboard"],
    remove_unused_columns=False,
    fp16=fp16,
)

# -----------------------------
# Trainer
# -----------------------------
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=nepali_train["train"],
    eval_dataset=nepali_train["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

/tmp/ipython-input-2157445051.py:128: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7267


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
50,2.373200
100,1.405700
150,1.033900
200,0.932300
250,0.862900
300,0.855600
350,0.781700
400,0.750900
450,0.714600
500,0.612200


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=1000, training_loss=0.7454318752288819, metrics={'train_runtime': 3440.5565, 'train_samples_per_second': 4.65, 'train_steps_per_second': 0.291, 'total_flos': 4.6319330304e+18, 'train_loss': 0.7454318752288819, 'epoch': 6.668896321070234})

In [ ]:
import evaluate
import torch

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_from_disk
import evaluate

# Load the dataset and model
nepali_train = load_from_disk("/content/drive/MyDrive/nepali_whisper_processed")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="ne", task="transcribe")

# Load your trained model (use the latest checkpoint)
model_path = f"/content/drive/MyDrive/whisper-small-lora-nepali/checkpoint-1000"  # or specific checkpoint
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model.eval()

# Load WER metric
wer_metric = evaluate.load("wer")

def predict_and_compare(model, processor, dataset, num_samples=10):
    """Compare model predictions with ground truth"""

    results = []
    wer_scores = []

    # Get random samples from test set
    test_dataset = dataset["test"]
    indices = np.random.choice(len(test_dataset), min(num_samples, len(test_dataset)), replace=False)

    print(" Comparing Predictions vs Ground Truth:\n")
    print("=" * 100)

    for i, idx in enumerate(indices):
        # Convert numpy index to standard integer
        sample = test_dataset[int(idx)]

        # Get input features
        input_features = torch.tensor(sample["input_features"]).unsqueeze(0)

        # Generate prediction
        with torch.no_grad():
            predicted_ids = model.generate(input_features, max_length=225)

        # Decode predictions and labels
        prediction = processor.tokenizer.decode(predicted_ids[0], skip_special_tokens=True)

        # Get ground truth (remove -100 padding)
        label_ids = sample["labels"]
        label_ids = [token_id for token_id in label_ids if token_id != -100]
        ground_truth = processor.tokenizer.decode(label_ids, skip_special_tokens=True)

        # Calculate WER for this sample
        wer = wer_metric.compute(predictions=[prediction], references=[ground_truth])
        wer_scores.append(wer)

        results.append({
            'prediction': prediction,
            'ground_truth': ground_truth,
            'wer': wer
        })

        # Print comparison
        print(f"Sample {i+1}:")
        print(f"Ground Truth: {ground_truth}")
        print(f"Prediction:   {prediction}")
        print(f"WER: {wer:.2%}")
        print("-" * 80)

    # Overall statistics
    avg_wer = np.mean(wer_scores)
    print(f"\nOverall Average WER: {avg_wer:.2%}")
    print(f"WER Range: {np.min(wer_scores):.2%} - {np.max(wer_scores):.2%}")

    return results, wer_scores

# Run prediction comparison
results, wer_scores = predict_and_compare(model, processor, nepali_train, num_samples=10)

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


 Comparing Predictions vs Ground Truth:



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sample 1:
Ground Truth: साथी तिमी बेनी बजार गएको छौ?
Prediction:   साति तिमी बेनी बजार गयागो छुु ।
WER: 66.67%
--------------------------------------------------------------------------------
Sample 2:
Ground Truth: काम र पढाइले गर्दा नै हो धेरै त ।
Prediction:    काम्रा पडाँ ब्राँ ब्राँ ब्राँ ग्र्दा नेँ बुदिरिता ।
WER: 88.89%
--------------------------------------------------------------------------------
Sample 3:
Ground Truth: अनाज अन्न र ओरिजा सातिभाको बीउ
Prediction:   अनाज अन्न्ड र उरिजा सातिबहाको व्यु
WER: 66.67%
--------------------------------------------------------------------------------
Sample 4:
Ground Truth: पोखरा त नेपालकै सफा सहर हो नि,सायद धुलो कम नै होला ।
Prediction:   पुख्रात नेपालके सफा सहरहो नि साया दुलुकम्ने अला ।
WER: 83.33%
--------------------------------------------------------------------------------
Sample 5:
Ground Truth: सोडियम क्लोराइडको खनिज अवस्था
Prediction:   सोडियम क्लोराइटको खानिज अब्ष्टा
WER: 75.00%
-----------------------------------------------

Original model before finetuning

In [ ]:
import evaluate
import torch
import numpy as np # Import numpy

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

def compare_with_base_model(num_samples=5):
    """Compare your fine-tuned model with the base Whisper model"""

    # Load base model
    base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
    base_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
    base_model.eval()

    test_dataset = nepali_train["test"]
    indices = np.random.choice(len(test_dataset), min(num_samples, len(test_dataset)), replace=False)

    print(" Comparison: Fine-tuned vs Base Model")
    print("=" * 100)

    for i, idx in enumerate(indices):
        # Convert numpy index to standard integer
        sample = test_dataset[int(idx)] # Corrected here
        input_features = torch.tensor(sample["input_features"]).unsqueeze(0)

        # Your fine-tuned model prediction
        with torch.no_grad():
            fine_tuned_pred = model.generate(input_features, max_length=225)
        ft_prediction = processor.tokenizer.decode(fine_tuned_pred[0], skip_special_tokens=True)

        # Base model prediction
        with torch.no_grad():
            base_pred = base_model.generate(input_features, max_length=225)
        base_prediction = base_processor.tokenizer.decode(base_pred[0], skip_special_tokens=True)

        # Ground truth
        label_ids = sample["labels"]
        label_ids = [token_id for token_id in label_ids if token_id != -100]
        ground_truth = processor.tokenizer.decode(label_ids, skip_special_tokens=True)

        # Calculate WER
        ft_wer = wer_metric.compute(predictions=[ft_prediction], references=[ground_truth])
        base_wer = wer_metric.compute(predictions=[base_prediction], references=[ground_truth])

        print(f"Sample {i+1}:")
        print(f"Ground Truth: {ground_truth}")
        print(f"Fine-tuned:   {ft_prediction} (WER: {ft_wer:.2%})")
        print(f"Base Model:   {base_prediction} (WER: {base_wer:.2%})")
        # Avoid division by zero if base_wer is 0
        improvement = ((base_wer - ft_wer) / base_wer * 100) if base_wer != 0 else float('inf')
        print(f"Improvement:  {improvement:+.1f}%")
        print("-" * 80)

# Uncomment to run comparison
compare_with_base_model(num_samples=3)

 Comparison: Fine-tuned vs Base Model
Sample 1:
Ground Truth: यो गाउँको सिमाना पातेमेला जामनेपानी छाम कर्णाली नदीसँग जोडिएको छ
Fine-tuned:   यो गएउगाउँगो सिमाना पाटे मेला जामणे पानि चाम् कर्नाली नदी सग्दिएको छ (WER: 90.00%)
Base Model:    यो गवको शिमाना पाते मेला जामने पानी चाम करनाली नदी संगा जोडी एको चाए. (WER: 130.00%)
Improvement:  +30.8%
--------------------------------------------------------------------------------
Sample 2:
Ground Truth: उहाँलाई चमत्कारिक व्यक्तित्वको रूपमा लिने गरिन्छ
Fine-tuned:   वाहालाई चम्ब्ट कारिक बेक्तिक्तोको रूपमा लिने गरिन्छ (WER: 66.67%)
Base Model:    वहा ले चवमत करिक बेक्तिख तोको रुप माल दिने गरिन सा. (WER: 183.33%)
Improvement:  +63.6%
--------------------------------------------------------------------------------
Sample 3:
Ground Truth: वालग्रिन डट् कम् यसको सुरुवातका वर्षमै प्रगतिको बाटामा लागेको थियो
Fine-tuned:   वाल्ग्रिन्ड ड्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्

For enlish language

In [ ]:
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load your fine-tuned model
model_path = f"/content/drive/MyDrive/whisper-small-lora-nepali/checkpoint-1000"
model = WhisperForConditionalGeneration.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Test English sentences
english_tests = [
    "Hello, how are you doing today?",
    "The weather is beautiful outside.",
    "I would like to order a coffee please.",
]

print("🧪 Testing with Language Forcing")
print("=" * 60)

for i, text in enumerate(english_tests, 1):
    dummy_audio = np.random.randn(16000 * 3)

    input_features = processor(
        dummy_audio,
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features

    # Force English language during generation
    forced_decoder_ids = processor.get_decoder_prompt_ids(
        language="en",
        task="transcribe"
    )

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            max_length=225,
            forced_decoder_ids=forced_decoder_ids  # Force English output
        )

    prediction = processor.decode(predicted_ids[0], skip_special_tokens=True)

    print(f"{i}. Expected: '{text}'")
    print(f"   Got:      '{prediction}'")
    print()

🧪 Testing with Language Forcing
1. Expected: 'Hello, how are you doing today?'
   Got:      ' Ssshh!'

2. Expected: 'The weather is beautiful outside.'
   Got:      ' Ssshh!'

3. Expected: 'I would like to order a coffee please.'
   Got:      '.'



#Finetuning further from the checkpoint

In [3]:
from datasets import load_dataset

dataset = load_dataset("spktsagar/openslr-nepali-asr-cleaned", name="cleaned", split='train')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for spktsagar/openslr-nepali-asr-cleaned contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/spktsagar/openslr-nepali-asr-cleaned.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] yes


Generating train split:   0%|          | 0/157905 [00:00<?, ? examples/s]

In [4]:
print(f"Loaded {len(dataset)} training samples.")
print(dataset)
print(dataset[0])  # Preview one example

Loaded 157905 training samples.
Dataset({
    features: ['utterance_id', 'speaker_id', 'utterance', 'transcription', 'num_frames'],
    num_rows: 157905
})
{'utterance_id': '4aa1fdca33', 'speaker_id': '6a6d1', 'utterance': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1fd1856a856596cbfeb72ce5285abab35ed69b2538c376e9d4ad529a3eee6abf/cleaned/asr_nepali/data/4a/4aa1fdca33.flac', 'array': array([-3.23486328e-03, -2.38037109e-03,  6.43920898e-03, ...,
        9.15527344e-05,  9.15527344e-05, -5.49316406e-04]), 'sampling_rate': 16000}, 'transcription': '००७ मिलको दूरीमा', 'num_frames': 43200}


In [5]:
# -------------------------------------------------
# 1. Imports
# -------------------------------------------------
import os
import torch
import numpy as np
from datasets import Dataset, load_dataset, DatasetDict
from transformers import (
    AutoFeatureExtractor,
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import evaluate
import jiwer

In [6]:
split = dataset.train_test_split(test_size=0.05, seed=42)
dataset = DatasetDict({
    "train": split["train"],
    "validation": split["test"],
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['utterance_id', 'speaker_id', 'utterance', 'transcription', 'num_frames'],
        num_rows: 150009
    })
    validation: Dataset({
        features: ['utterance_id', 'speaker_id', 'utterance', 'transcription', 'num_frames'],
        num_rows: 7896
    })
})


In [7]:
checkpoint_dir = "/content/drive/MyDrive/whisper-small-lora-nepali/checkpoint-1000"

In [8]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from peft import PeftModel

base_model = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(base_model)

# Load base Whisper
model = WhisperForConditionalGeneration.from_pretrained(
    base_model, device_map="auto", torch_dtype="auto"
)

# Attach LoRA checkpoint (your previous fine-tuning)
model = PeftModel.from_pretrained(model, checkpoint_dir)

print("Loaded model with previous LoRA fine-tuning")


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Loaded model with previous LoRA fine-tuning


In [9]:
feature_extractor = processor.feature_extractor
sampling_rate = feature_extractor.sampling_rate

In [ ]:
import librosa
import soundfile as sf
import numpy as np
import os

sampling_rate = 16000 # Define sampling_rate before it is used
DUMMY_DURATION = 0.1          # 0.1 s of silence → 1600 samples @ 16 kHz
DUMMY_AUDIO   = np.zeros(int(DUMMY_DURATION * sampling_rate), dtype=np.float32)



def _dummy_example():
    """Return a valid Whisper example for a corrupted file."""
    input_features = feature_extractor(
        DUMMY_AUDIO, sampling_rate=sampling_rate, return_tensors="pt"
    ).input_features[0]

    # empty transcription → all -100 (ignored by the loss)
    labels = processor.tokenizer("", return_tensors="pt").input_ids[0]
    labels = torch.where(labels == processor.tokenizer.pad_token_id, -100, labels)

    return {"input_features": input_features, "labels": labels}


def prepare_example(example):
    """
    Convert one dataset row → Whisper input_features + labels.
    If anything fails we return a dummy example (never None).
    """
    try:
        # ------------------------------------------------------------------
        # 1. Load audio – the dataset uses the key "utterance"
        # ------------------------------------------------------------------
        audio = example["utterance"]
        if audio is None or audio.get("array") is None:
            raise ValueError("Audio dict is empty")

        array = np.asarray(audio["array"], dtype=np.float32)
        sr    = audio["sampling_rate"]

        # ------------------------------------------------------------------
        # 2. Resample to 16 kHz (OpenSLR is already 16 kHz, but be safe)
        # ------------------------------------------------------------------
        if sr != sampling_rate:
            array = librosa.resample(array, orig_sr=sr, target_sr=sampling_rate)

        # ------------------------------------------------------------------
        # 3. Feature extraction
        # ------------------------------------------------------------------
        input_features = feature_extractor(
            array, sampling_rate=sampling_rate, return_tensors="pt"
        ).input_features[0]

        # ------------------------------------------------------------------
        # 4. Tokenize transcription (fallback to empty string)
        # ------------------------------------------------------------------
        text = example.get("transcription", "") or ""
        labels = processor.tokenizer(text, return_tensors="pt").input_ids[0]
        labels = torch.where(labels == processor.tokenizer.pad_token_id, -100, labels)

        return {"input_features": input_features, "labels": labels}

    # ------------------------------------------------------------------
    # ANY error → log + dummy example
    # ------------------------------------------------------------------
    except (sf.LibsndfileError, OSError, ValueError, Exception) as e:
        uid = example.get("utterance_id", "unknown")
        print(f"Warning: Skipping utterance {uid}: {e}")
        return _dummy_example()


# ------------------------------------------------------------------
# 8. Apply the map (single-process, no batching)
# ------------------------------------------------------------------
processed = dataset.map(
    prepare_example,
    remove_columns=dataset["train"].column_names,   # keep only input_features & labels
    num_proc=1,
    batched=False,
    desc="Preprocessing audio & text",
)

# ------------------------------------------------------------------
# 9. Remove dummy examples (they have zero-length labels)
# ------------------------------------------------------------------
def _is_real(example):
    # dummy label length = 1 (empty string → one token → replaced by -100)
    return len(example["labels"]) > 1

clean_train = processed["train"].filter(_is_real, desc="Drop dummy train")
clean_val   = processed["validation"].filter(_is_real, desc="Drop dummy val")

dataset = DatasetDict({"train": clean_train, "validation": clean_val})


# Choose a directory (you can change this)
SAVE_DIR = "/content/drive/MyDrive/openslr_nepali_whisper_processed"

# Create directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

# Save the clean DatasetDict
dataset.save_to_disk(SAVE_DIR)

print("\nPreprocessing finished!")
print(f"Train samples : {len(dataset['train'])}")
print(f"Val   samples : {len(dataset['validation'])}")

Preprocessing audio & text:   0%|          | 0/150009 [00:00<?, ? examples/s]